# 01 Visualizing KITTI

The format of each Velodyne file is 32-bit floats representing an array of [x, y, z, b] points, where "b" is brightness or irradiance. There is otherwise no delimiter. 

In [ ]:
#|default_exp kitti.vis

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Union, Iterable
from pathlib import Path
from os import PathLike, environ

from pathlib import Path
import numpy as np
import open3d as o3d

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import plotly.graph_objects as go


In [ ]:
#| export
KITTI_ROOT:Path = environ.get("KITTI_ROOT", Path("/media/usbhd4tb/datasets/kitti"))

# Documentation: https://www.open3d.org/docs/release/python_api/open3d.ml.torch.datasets.KITTI.html
ds:o3d.ml.datasets.KITTI = o3d.ml.datasets.KITTI(KITTI_ROOT)

In [ ]:
ds.get_label_to_names()

{0: 'Pedestrian',
 1: 'Cyclist',
 2: 'Car',
 3: 'Van',
 4: 'Person_sitting',
 5: 'DontCare'}

In [ ]:
len(ds.get_split("train"))

3712

In [ ]:
#| export
def loadVeloAsPcd(fname: Union[str, PathLike])->o3d.geometry.PointCloud:
    points          = o3d.ml.datasets.KITTI.read_lidar(fname)
    cloud           = o3d.geometry.PointCloud()
    cloud.points    = o3d.utility.Vector3dVector(points[:, :3].astype(float))
    cloud.estimate_normals(search_param = o3d.geometry.KDTreeSearchParamHybrid(radius = 0.1, max_nn = 30))
    return cloud

In [ ]:
num:str                         = "000900"
fname                           = KITTI_ROOT/"training"/"velodyne_reduced"/(num+".bin")
cloud:o3d.geometry.PointCloud   = loadVeloAsPcd(fname)
cloud

PointCloud with 19913 points.

In [ ]:
#| export
def plotCloud(
        cloud: o3d.geometry.PointCloud, 
        boxes:Iterable[o3d._ml3d.datasets.kitti.Object3d] = [])->go.Figure:
    
    points = np.asarray(cloud.points)
    
    colors = None
    if cloud.has_colors():
        colors = np.asarray(cloud.colors)
    elif cloud.has_normals():
        colors = (0.5, 0.5, 0.5) + np.asarray(cloud.normals) * 0.5
    else:
        colors = np.zeros((3, 1), dtype=np.float64) 
        cloud.paint_uniform_color(colors)  
    

    # Add the point cloud
    data = [go.Scatter3d(
        x=points[:, 0], y=points[:, 1], z=points[:, 2],
        mode = 'markers',
        marker = {"size": 1, "color": colors}
    )]

    # Add bounding boxes
    for box in boxes:
        corners = box.generate_corners3d()
        x, y, z = corners[:, 0], corners[:, 1], corners[:, 2]
        mesh = go.Mesh3d(
            x=x, y=y, z=z,
            color='rgba(0, 255, 0, 0.5)',  # Semi-transparent green
            opacity=0.5
        )
        data.append(mesh)

    # Global figure settings
    layout = {
        "scene": {
            "xaxis": {"visible": False},
            "yaxis": {"visible": False},
            "zaxis": {"visible": False}
        }
    }

    fig = go.Figure(data=data, layout=layout)
    return fig

In [ ]:
calib = ds.read_calib(KITTI_ROOT/"training"/"calib"/(num+".txt"))
calib

{'world_cam': array([[ 2.34773921e-04,  1.04494076e-02,  9.99945343e-01,
          0.00000000e+00],
        [-9.99944150e-01,  1.05653545e-02,  1.24365499e-04,
          0.00000000e+00],
        [-1.05634769e-02, -9.99889612e-01,  1.04513029e-02,
          0.00000000e+00],
        [-2.79681687e-03, -7.51087889e-02, -2.72132814e-01,
          1.00000000e+00]], dtype=float32),
 'cam_img': array([[7.215377e+02, 0.000000e+00, 0.000000e+00, 0.000000e+00],
        [0.000000e+00, 7.215377e+02, 0.000000e+00, 0.000000e+00],
        [6.095593e+02, 1.728540e+02, 1.000000e+00, 1.000000e+00],
        [4.485728e+01, 2.163791e-01, 2.745884e-03, 0.000000e+00]],
       dtype=float32)}

In [ ]:
label = KITTI_ROOT/"training"/"label_2"/(num+".txt")
label = ds.read_label(label, calib)

label[0].generate_corners3d()

array([[-1.70254446,  1.65      , 40.59824648],
       [-3.29049358,  1.65      , 40.51751505],
       [-3.51745553,  1.65      , 44.98174939],
       [-1.9295064 ,  1.65      , 45.06248082],
       [-1.70254446,  0.24      , 40.59824648],
       [-3.29049358,  0.24      , 40.51751505],
       [-3.51745553,  0.24      , 44.98174939],
       [-1.9295064 ,  0.24      , 45.06248082]])

In [ ]:
fig = plotCloud(cloud)
fig.show()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()